# SHAP

SHAP(SHapley Additive exPlanations)는 섀플리 값(Shapley value)과 피처 간 독립성을 핵심 아이디어로 사용하는 XAI 기법이다.

섀플리 값은 전체 성과를 창출하는 데 각 참여자가 얼마나 공헌했는지를 수치로 표현할 수 있다.

섀플리 값에 의하면, 각 사람의 기여도는 그 사람의 기여도를 제외했을 때 전체 성과의 변화 정도로 나타낼 수 있다.

섀플리 값은 모델이 표현할 수 있는 모든 조합과 특정 피처를 제외한 피처의 조합을 빼서 평균을 낸다.

SHAP는 모델의 출력을 각 피처의 기여도로 분해한다.

이때 섀플리 값은 음수일 수 있으며, 이 경우 특정 피처가 예측에 부정적인 영향을 미치고 있다고 해석할 수 있다.

SHAP는 피처 중요도와 유사해 보이지만, 피처 간 의존성까지 고려해서 모델 영향력을 계산한다.

또한, SHAP는 학습된 모델에 대해서만 설명할 수 있다. 따라서 피처의 추가와 삭제가 빠른 모델을 설명하기에 적합하지 않다.

예를 들어, 집값 모델을 비유로 세 가지 XAI 알고리즘을 비교해본다.

1. 피처 중요도는 집값을 예측하는 피처들 몇 개가 서로 의존적이라면 피처 영향력이 잘못 계산될 수 있다.

2. 부분 의존성 플롯은 특정 피처가 변할 때 집값에 어떤 영향을 미치는지 시각화할 수 있다. 그러나 비교하고자 하는 피처가 많아지면 시각화할 수 없고(3차원까지), 피처 영향력이 과대 평가될 위험이 있다.

3. SHAP는 집값을 결정하는 기준이 어떻게 배분되고 있는지 균형 있게 해석한다. 그러나 아웃라이어의 등장에 허술한 해석을 내놓을 가능성이 높다.

1과 2는 모델의 관점에 대한 설명이고 3은 데이터 하나에 대한 설명을 구한다는 차이가 있다.

주택 문제를 해결해본다.

A씨는 주택 가격에 이상이 있어서 알고리즘을 돌려봤지만 문제 없다는 결론을 내렸다.

SHAP를 이용해 주택 시세를 결정하는 결정적인 요소를 찾아본다.

In [2]:
import shap
from sklearn.model_selection import train_test_split

X, y = shap.datasets.boston()
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size = 0.2,
                                                   random_state = 1)
X_train[:10]

/root/anaconda3/envs/xai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/root/anaconda3/envs/xai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/root/anaconda3/envs/xai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/root/anaconda3/envs/xai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
42,0.14150,0.0,6.91,0.0,0.448,6.169,6.6,5.7209,3.0,233.0,17.9,383.37,5.81
58,0.15445,25.0,5.13,0.0,0.453,6.145,29.2,7.8148,8.0,284.0,19.7,390.68,6.86
385,16.81180,0.0,18.10,0.0,0.700,5.277,98.1,1.4261,24.0,666.0,20.2,396.90,30.81
78,0.05646,0.0,12.83,0.0,0.437,6.232,53.7,5.0141,5.0,398.0,18.7,386.40,12.34
424,8.79212,0.0,18.10,0.0,0.584,5.565,70.6,2.0635,24.0,666.0,20.2,3.65,17.16
160,1.27346,0.0,19.58,1.0,0.605,6.250,92.6,1.7984,5.0,403.0,14.7,338.92,5.50
185,0.06047,0.0,2.46,0.0,0.488,6.153,68.8,3.2797,3.0,193.0,17.8,387.11,13.15
101,0.11432,0.0,8.56,0.0,0.520,6.781,71.3,2.8561,5.0,384.0,20.9,395.58,7.67
268,0.54050,20.0,3.97,0.0,0.575,7.470,52.6,2.8720,5.0,264.0,13.0,390.30,3.16
173,0.09178,0.0,4.05,0.0,0.510,6.416,84.1,2.6463,5.0,296.0,16.6,395.50,9.04


In [4]:
# 방의 개수와 집값 간의 관계를 산점도로 그리는 코드

import matplotlib.pyplot as plt
import matplotlib
%matplotlib
matplotlib.style.use('ggplot')

fig, ax1 = plt.subplots(1,1,figsize=(12, 6))

ax1.scatter(X["RM"], y, color = 'black', alpha=0.6)

ax1.set_title("Relation # of Rooms with MEDV")
ax1.set_xlim(2.5,9)
ax1.set_xlabel('RM')
ax1.set_ylim(0, 55)
ax1.set_ylabel("MEDV \n Price $1,000")

Using matplotlib backend: TkAgg


Text(0, 0.5, 'MEDV \n Price $1,000')

방 개수가 증가할 때 주택 매매 가격이 상승하는 경향을 보인다.

그러나 예외도 존재했따. 방이 9개지만 방 4개짜리 집보다 저렴하다.

비정상적으로 보이는 어떤 값은 실제로는 잘 보이지 않는 합리적 피처의 조합으로 형성됐을 가능성이 높다.

13개의 복잡한 피처가 맞물려 집값을 결정할 때 이것을 합리적으로 설명할 수 있는 초평면(hyperplane)을 찾아야 한다.

In [7]:
# 선형 모델을 이용해서 방 개수와 주택 가격 간의 관계를 구하는 코드

from sklearn import linear_model
import pandas as pd

linear_regression = linear_model.LinearRegression()
linear_regression.fit(X=pd.DataFrame(X_train['RM']), y=y_train)
prediction = linear_regression.predict(X=pd.DataFrame(X_test['RM']))

print('a value: ', linear_regression.intercept_)
print('b value: ', linear_regression.coef_)
print('MEDV = {:.2f} * RM {:.2f}'.format(linear_regression.coef_[0],
                                        linear_regression.intercept_))

a value:  -32.39552264560753
b value:  [8.76050748]
MEDV = 8.76 * RM -32.40


In [10]:
fig, ax1 = plt.subplots(1,1,figsize=(12, 6))

ax1.scatter(X_train["RM"], y_train, color = 'black', alpha=0.4, label = 'data')
ax1.scatter(X_test["RM"], y_test, color = '#993299', alpha=0.4, label = 'data')

ax1.set_title("Relation # of Rooms with MEDV")
ax1.set_xlim(2.5,9)
ax1.set_xlabel('RM')
ax1.set_ylim(0, 55)
ax1.set_ylabel("MEDV \n Price $1,000")

ax1.plot(X_test['RM'], prediction, color = 'purple', alpha = 1,
        linestyle='--', label = 'linear regression line')

ax1.legend()

회귀 분석은 데이터를 전부 반영하지는 않지만, 데이터의 추세를 설명할 수 있다.

방 개수가 많아지면 집값이 비례해서 증가하는 경향을 보여준다.

회귀 분석은 필연적으로 예측값에 오차가 발생하기 때문에 성능을 측정하는 새로운 지표가 필요하다.

오차 지표는 RMSE를 사용한다.

In [13]:
from sklearn.metrics import mean_squared_error
import numpy as np
rmse = np.sqrt(mean_squared_error(y_test, prediction))

print("RMSE: %f"% (rmse))

RMSE: 6.383135


이 값만으로는 좋은 것인지, 나쁜 것인지 알 수 없다.

회귀 문제의 성능은 비교 성능 없이는 아무런 의미도 갖지 않는다.

In [14]:
# xgboost의 선형 회귀 모델로 주택 매매 가격을 예측하는 모델을 만들고 학습하는 코드

import xgboost

model = xgboost.XGBRegressor(objective='reg:linear')
model.fit(X_train, y_train)

preds = model.predict(X_test)

In [15]:
from sklearn.metrics import mean_squared_error
import numpy as np
rmse = np.sqrt(mean_squared_error(y_test, preds))

print("RMSE: %f"% (rmse))

RMSE: 2.818447


전체 데이터를 사용한 모델이 그렇지 않은 모델보다 오차가 적다. 

그러므로 전체 피처를 사용한 모델이 그렇지 않은 모델보다 더 잘 예측한다는 결론을 내릴 수 있다.

In [16]:
# SHAP의 설명체를 정의하고 섀플리 값을 계산하는 로직
shap.initjs()

# shap 값으로 모델의 예측을 설명하기
# 설명체는 LightGBM, CatBoost, scikit-learn 모델을 입력받을 수 있다.
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_train)

shap.force_plot(explainer.expected_value,
               shap_values[0,:],
               X_train.iloc[0,:])

설명체의 종류는 모델에 따라 다르게 선언할 수 있다.

일반적으로 딥러닝 모델을 위한 딥-설명체와 그 밖의 모델에 대한 트리 설명체, 

그리고 각 피처에 대한 가중치를 설정할 수 있는 커널 설명체가 있다.

집값 결정 모델은 각 피처가 개별적이고 피처 간 조합이 결과에 영향을 미치기 때문에 트리 설명체를 선언해서 분기를 관찰했다.

위 그림은 테스트 데이터 0번의 주택 매매 가격이 25.05로 추정되며, 

이때 집값 상승에 긍정적인 영향을 준 요소는 LSTAT가 가장 크다고 해석한다.

또한 집값 형성에 가장 부정적인 영향을 준 요소는 RM으로, 비슷한 조건의 주택에 비해 방의 개수가 상대적으로 적기 때문에

집값 형성에 부정적인 영향을 미쳤다고 해석할 수 있다.

이제 아웃라이어 집 한 채에 대한 집값 상승 요소를 파악해본다.

In [19]:
#259번 데이터에 대해서 방의 개수(RM)와 집 가격(MEDV)이 어떤 관계가 있는지 플롯으로 그리는 코드

fig, ax1 = plt.subplots(1,1,figsize=(12, 6))
idx = 259

ax1.scatter(X["RM"], y, color = 'black', alpha=0.4)
ax1.scatter(X_train["RM"].iloc[idx], y_train[idx], c = 'red',s = 150)

ax1.set_title("Relation # of Rooms with MEDV")
ax1.set_xlim(2.5,9)
ax1.set_xlabel('RM')
ax1.set_ylim(0, 55)
ax1.set_ylabel("MEDV \n Price $1,000")


Text(0, 0.5, 'MEDV \n Price $1,000')

259번 데이터는 방이 5개뿐인데도 집값이 50을 달성했다.

이제 259번 데이터에 대한 섀플리 영향력을 플롯으로 나타낸다.

In [20]:
shap.force_plot(explainer.expected_value,
               shap_values[259,:],
               X_train.iloc[259,:])

이 데이터에서는 집값을 결정하는 데 방 개수의 영향을 거의 받지 않는다는 것을 알 수 있다.

이 데이터의 집값이 높게 평가된 이유는 주요 업무지까지 거리가 가깝고(DIS),

주변 지역에 하위 계층 사람들이 적게 살면서(LSTAT),

범죄율도 극도로 낮다는(CRIM) 요인이 컸다.

즉, 주택 자체의 상태보다 주변 환경으로 평판이 좋은 건물이었다.

In [21]:
# 전체 데이터에 대한 섀플리 값을 플롯으로 그리는 코드

shap.force_plot(explainer.expected_value, shap_values, X_train)

In [22]:
# 전체 데이터에 대한 섀플리 값을 플롯으로 그리는 코드

shap.force_plot(explainer.expected_value, shap_values, X_train)

결과에 의하면, 범죄율은 수치가 존재하는 한 집값에 나쁜 영향을 미친다.

그러나 어떤 주택은 범죄율이 집값 하락에 적은 영향을 미치는가 하면,

어떤 주택은 하락에 결정적인 영향을 미치는 결과도 확인할 수 있다.

즉, 주택 간 범죄율이 똑같더라도 집값에 영향을 미치는 영향력은 다를 수 있다는 의미다.

In [23]:
# 방 개수 피처가 집값에 미치는 섀플리 영향도를 시각화하는 플롯

shap.dependence_plot('RM', shap_values, X_train)

붉은색 데이터는 다른 피처들보다 방 개수에 집값 변동이 큰 데이터군이고,

파란색 데이터는 방 개수가 다른 피처들에 비해 적은 영향을 미치는 경우다.

In [24]:
# 전체 피처들이 섀플리 값 결정에 어떻게 관여하고 있는지 시각화하는 코드
shap.summary_plot(shap_values, X_train)

붉은 색 점은 그 지점에 해당하는 행 피처가 집값을 결정하는 데 큰 영향을 미쳤음을 의미한다.

파란색 점은 행 피처가 집값을 결정하는 데 적은 영향을 미쳤다는 의미다.

LSTAT, RM, DIS는 분산이 크다. 붉은점도 이 세 개의 피처에 집중돼있다.

즉, LSTAT, RM, DIS 피처가 집값을 결정하는 데 큰 역할을 한다.

In [27]:
shap.summary_plot(shap_values, X_train, plot_type='bar')

In [26]:
# 피처 중요도
xgboost.plot_importance(model)

위 두 차트는 다른 결과를 낸다. 

왜 피처 중요도와 SHAP는 다른 결과를 냈을까? 그리고 이런 상황에서 어떤 XAI를 신뢰할 수 있을까?
<br><br>

피처 중요도는 에러가 높아지는 변인(음의 영향력)을 결과에 포함하지 않고, 

피처가 서로 의존적일 떄는 결과가 왜곡될 수 있다.

이런 단점에도 불구하고 피처 중요도는 측정하기 쉽고 알고리즘이 단순해서 모델의 간편한 이해를 위해 사용된다.
<br><br>

섀플리 값은 피처들이 서로 영향을 미칠 가능성을 고려한다. 

피처 간의 의존성을 고려하고 음의 영향력을 계산할 수 있다.

그러나 계산량이 많아서 측정 속도가 느리다.

<br>

집값은 피처별 의존성이 어느 정도 존재한다. 또한 집값을 결정하는 피처는 음의 영향을 주기도 한다.

따라서 이번 집값 예측 데이터는 피처 중요도 계산법보다 SHAP를 사용했을 때 정확한 영향력을 측정했다고 볼 수 있다.

SHAP의 장점

1. 피처가 서로 의존적일 때도 사용할 수 있다.

2. 피처가 모델에 미칠 음의 영향력까지도 계산할 수 있다.

SHAP의 단점

1. 데이터 수의 제곱에 비례해서 계산량이 증가한다.

2. 아웃라이어 데이터에 취약하다.